In [87]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [88]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from statsmodels.graphics.regressionplots import influence_plot
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from xgboost import XGBRegressor
%matplotlib inline

In [89]:
df_sales = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
df_sub = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')


def df_head(df):
    return df.head()

    
df_head(df_sales)

In [90]:
df_head(df_sub)

In [91]:
df_head(df_items)

In [92]:
df_head(df_shops)

In [93]:
df_head(df_test)

# Descriptive and exploratory data analysis


In [94]:
df_sales.describe().T

In [95]:
df_test.describe().T

In [96]:
df_sales['item_price']

In [97]:
df_sales[['shop_id','item_id','item_price','item_cnt_day']].corr()

In [98]:
df_items.describe().T

In [99]:
plt.figure(figsize=(8,5))
plt.hist(df_sales['item_id'])
plt.show

In [100]:
plt.figure(figsize=(8,5))
plt.hist(df_items['item_category_id'])
plt.show

In [101]:
plt.figure(figsize=(8,5))
plt.hist(df_sales['item_cnt_day'])
plt.show

In [102]:
df_items.groupby('item_category_id').count()

In [103]:
df_items.groupby('item_category_id').mean()

In [104]:
df_items['diff_col_of_item_id'] = df_items.groupby('item_category_id')['item_id'].max() - df_items.groupby('item_category_id')['item_id'].min()

df_items.head()

## train and test data set preparation and pre-processing

In [105]:
df_sales.head()

In [106]:
df_sales.isnull().sum()

In [107]:
df_sales.drop_duplicates(keep='first', inplace=True, ignore_index=True)

df_sales.head()

In [108]:
df_sales[df_sales['item_price'] <0]

In [109]:
df_sales.drop(df_sales[df_sales['item_cnt_day'] <0].index , inplace=True)
df_sales.drop(df_sales[df_sales['item_price'] <0].index , inplace=True)

df_sales.shape

## outliers removal


In [110]:
Q1 = np.percentile(df_sales['item_price'], 25.0)
Q3 = np.percentile(df_sales['item_price'], 75.0)

IQR = Q3 - Q1

df_sub1 = df_sales[df_sales['item_price'] > Q3 + 1.5*IQR]
df_sub2 = df_sales[df_sales['item_price'] < Q1 - 1.5*IQR]

df_sales.drop(df_sub1.index, inplace=True)

df_sales.shape

In [111]:
df_sales['date_block_num'].unique()

In [112]:
df_sales.groupby('date_block_num')['item_id'].mean()

In [113]:
price = round(np.array(df_sales.groupby('date_block_num')['item_price'].mean()).mean(),2)
print(price)

In [114]:
dict(round(df_sales.groupby('date_block_num')['item_price'].mean(),4))

In [115]:
df_sales.head()

In [116]:
df_test.head()

# Feature engineering

## FE workflow

here create columns with mean price by date block in train and test dataset, remove item price from train

remove data_ block column from train data set

create new cloumns with mean price per shop id for both train and test dataset

merge df_items table with train and test dataset on item id and create new column item category

In [117]:
#df_sales.drop('mean_price_data_block', inplace=True, axis=1)

replace_dict = dict(round(df_sales.groupby('date_block_num')['item_price'].mean(),2))

In [118]:
df_sales['date_block_num'] = df_sales['date_block_num'].replace(replace_dict)

df_train = df_sales.copy()
df_train.drop(['date','item_price'], axis=1, inplace=True)
df_train.rename(columns = {'date_block_num':'mean_price_by_column'}, inplace=True)
df_train.head()

In [119]:
mean_price = np.array(df_sales.groupby('date_block_num')['item_price'].mean()).mean()
mean_price

In [120]:
df_test.shape

In [121]:
df_train.shape

In [122]:
#df_test.drop('ID', inplace=True, axis=1)
df_test.head()
com_df = pd.concat([df_train,df_test])

com_df['mean_price_by_column'] = com_df['mean_price_by_column'].fillna(value=price)
com_df['item_cnt_day'] = com_df['item_cnt_day'].fillna(value=0)

test_df = com_df[com_df['item_cnt_day'] == 0]
train_df = com_df[com_df['item_cnt_day'] != 0]

In [123]:
test_df.shape

In [124]:
testdf = test_df.copy()

testdf.drop('ID', inplace=True, axis=1)
testdf.drop('item_cnt_day', inplace=True, axis=1)
testdf

In [125]:
traindf = train_df.copy()

traindf.drop('ID', inplace=True, axis=1)

In [126]:
traindf.head()

## train data and test data for modelling and evalution

In [127]:
#test_df.drop('item_cnt_day', inplace=True, axis=1)
testdf['item_id'] = (testdf['item_id'] - testdf['item_id'].mean())/testdf['item_id'].std()
testdf.head()

In [128]:
traindf['item_id'] = (traindf['item_id'] - traindf['item_id'].mean())/traindf['item_id'].std()
traindf.head()

## Model Creation
### Model 1

In [129]:
X = traindf.loc[:,['mean_price_by_column','shop_id','item_id']]
y = traindf.loc[:,'item_cnt_day']

In [130]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y,train_size=0.8, random_state= 42)

model1 = LinearRegression()

model1.fit(X_train,y_train)

print("regression coefficients are: " , model1.coef_)

y_pred = model1.predict(X_valid)


MSE = mean_squared_error(y_valid,y_pred)
MAE = mean_absolute_error(y_valid,y_pred)
R2  = r2_score(y_valid,y_pred)

print("MSE: ", MSE)
print("MAE: ", MAE)
print("R2: ", R2)

# Model 2

In [131]:
# addding constant as statsmodels api does not include it!
X_new = X.copy()
X_new = sm.add_constant(X_new)
test_df_new = test_df.copy()
test_df_new = sm.add_constant(test_df_new)
X_train_new, X_valid_new, y_train_new, y_valid_new = train_test_split(X_new, y,train_size=0.8, random_state= 42)

In [132]:
#using statsmodel api

stats_model = sm.OLS(y_train_new, X_train_new)
stat_fit = stats_model.fit()
print("Model coeffieciants: ", stat_fit.params)

print("\nModel summary: ", stat_fit.summary2())

Here P value is less than 0.05 indicates that model is statistically significant

F-stat is also very low

R2 is almost 17%, that explains independent variables explain 17% of variation in dependent variable item count

# Residual Analysis

In [133]:
model_residual = stat_fit.resid
probplot = sm.ProbPlot(model_residual)
plt.figure(figsize=(8,8))
probplot.ppplot(line = '45')
plt.title("Rsidual analysis of model 2")
plt.show()

* theoritical probabilties and sample probabilities are not co-relating with each other, it implies that resduals are not following normal distribution.

* model is not a good fit to data as data is polynomial.

# Test of Homoscedasticity

In [134]:
def get_std_val(vals):
    return (vals - vals.mean())/vals.std()

plt.figure(figsize=(8,8))
plt.scatter(get_std_val(stat_fit.fittedvalues), get_std_val(model_residual))
plt.title("Residual plot")
plt.xlabel("predicted values")
plt.ylabel("Residuals")
plt.show()

* Funnel shape pattern is observed and also we can identify that there is couple of outliers in actual values of y.

# Outlier analysis

In [135]:
traindf['z_score_item'] = zscore(traindf['item_cnt_day'])

In [136]:
#outliears in y variable

traindf[(traindf['z_score_item'] > 3.0) | (traindf['z_score_item'] < -3.0)]

> In this there are total of 9971 rows that are having extreme item sales which resulted in higher residuals

# Cook's distance

In [138]:
#item_influence = stat_fit.get_influence()
#(c, p) = item_influence.cooks_distance

#plt.stem(np.arange(len(X_train)),
        # np.round(c, 3),
       #  markerfmt=',')
#plt.title("Cook's distance")
#plt.xlabel("row index")
#plt.ylabel('Cook\'s distance')
#plt.show

> Using influence method we can plot cooks distance to find which observance has a most influence on output variable

# Leverage Values

In [140]:
#fig, ax = plt.subplots(figsize=(8,6))
#influence_plot(stat_fit, ax=ax)
#plt.title("Influence plot")
#plt.show()

In [141]:
pred_y = stat_fit.predict(X_valid_new)

r2 = r2_score(y_valid_new,pred_y)
mse = mean_squared_error(y_valid_new,pred_y)
mae = mean_absolute_error(y_valid_new,pred_y)

print(r2)
print(mse)
print(mae)

In [142]:
#model_sub = stat_fit.predict(testdf)

#sub_dff = pd.DataFrame(data= {'ID':np.array(df_test['ID']),'item_cnt_month':model_sub})

#sub_dff.to_csv('submission.csv', index=False) 

# Calculating prediction intervals

In [ ]:
pred_y = stat_fit.predict(X_valid_new)

_, pred_y_low, pred_y_high = wls_prediction_std( stat_fit, 
                                                X_valid_new, 
                                                alpha = 0.1)

pred_int_df = pd.DataFrame({'item_id_z': X_valid['item_id'],
                            'pred_y': np.array(pred_y),
                            'Pred_y_low': pred_y_low,
                             'Pred_y_high': pred_y_high
                           })

pred_int_df.head(10)

> Using statsmodels wls_prediction_std method we have calculated prediction interval for each predicted value of y.

# Final Model

In [143]:
model3 = XGBRegressor(n_estimators=50,
                      max_depth=3,
                      learning_rate = 0.01)

model3.fit(X_train, y_train)

prey = model3.predict(X_valid)

sq_error = mean_squared_error(y_valid, prey)

print(sq_error)

In [144]:
model3_sub = model3.predict(testdf)

sub_dff2 = pd.DataFrame(data= {'ID':np.array(df_test['ID']),'item_cnt_month':model3_sub})

sub_dff2.to_csv('submission.csv', index=False) 